# Idea: Maybe we can skip the untrained layers completely.
## Answer: No

In [48]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, modeling_utils, GPT2Config, modeling_gpt2, GPT2Model, GPT2PreTrainedModel, GPT2Config
import pickle
from utility import *

In [34]:
from collections import OrderedDict

In [35]:
config = GPT2Config(n_layer = 6)
model = GPT2LMHeadModel(config)

In [36]:
weights = torch.load("pytorch_model.bin")

In [37]:
out = OrderedDict()
chars = ["6","7","8","9","10","11"]
for c in weights.keys():
    var = True
    for x in chars:
        if x in c:
            var = False
    if var == True:
        out[c] = weights[c]
        


In [38]:
model.load_state_dict(out)

<All keys matched successfully>

In [43]:
%%capture
lam = open("LAMBADA/lambada_control_test_data_plain_text.txt","r+",encoding="utf-8")
lam = lam.read()
lam = lam.splitlines()
lamT = []
for x in lam:
    lamT.append(tokenizer.encode(x))

In [44]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
GPT_org = GPT2LMHeadModel.from_pretrained("gpt2")

In [45]:
step1 = pickle.load(open("LAMBADA/save_preds_fine_1.p","rb"))
step1_corpus = lamT

In [46]:
%%capture
out= []
for x in range(len(step1)):
    final = step1[x][0]
    sentence = tokenizer.decode(lamT[x][:-1])
    out.append(sentence + final)

In [49]:
eos = split_eos(out)

In [50]:
model = model.to("cuda")

In [52]:
condition = "The Technology Report empowers or enlightens. ==== The Technology Report empowers or enlightens."

In [54]:
%%capture
predictions = []
for x in eos[:1]: 
    inp = condition + " " + x[-1] + "===="
    inp = tokenizer.encode(inp) 
    predictions.append(generation(model,tokenizer,inp))
    
pickle.dump(predictions,open("save_preds_frozen_test.p","wb"))

In [55]:
oof = pickle.load(open("save_preds_frozen_test.p","rb"))

In [58]:
eos[0]

["there weren't many people this high up, which made my search very easy.",
 "they weren't here.",
 "`` let's go up one more level, '' i said, disappointed.",
 '`` hurry.',
 "'' we tread the rest of the way up the stairs and stood at the edge of the highest level.",
 'this far up, the music was barely a whisper']

In [56]:
oof

["======== \n“I'm not really looking at any particular particular particular person who would be able accessibly, but rather than give any significant amount of work to the same exacting levels of the same exacting action, which we've been seen in the same exacting stroke. And then we've got a lot of time to go back home to where we've been shownered cleagrificity.======= The same thing happened in order to be a different part of the same thing.<|endoftext|>"]

# Nope.